In [9]:
import os
import glob
import pandas as pd
import numpy as np
import funcs_for_proc
from transformator import (
    relative_concentration, get_accepted_maximum_value,
    get_all_relative_concentration)
import warnings
warnings.simplefilter("ignore", UserWarning)

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
from itertools import product
from tqdm import tqdm

In [10]:
data = funcs_for_proc.get_data_from_csv('final_data/data_train/data_train', 'csv','class/train.csv')

In [11]:
data_with_egde_val=pd.DataFrame(columns=['H2', 'CO', 'C2H4', 'C2H2','H2v', 'COv', 'C2H4v', 'C2H2v','H2a', 'COa', 'C2H4a', 'C2H2a','status'])
data_for_an=data
for i,file in enumerate(data_for_an):
    insertdata=[]
    datav=file[2].diff()
    dataa=datav.diff()
    vmax_last=list(datav.iloc[len(datav)-1].values)
    amax_last=list(dataa.iloc[len(dataa)-1].values)
    con=list(file[2].iloc[len(file[2])-1].values)
    status=[file[1]]
    insertdata=con+vmax_last+amax_last+status
    data_with_egde_val.loc[i]=insertdata
    
    

In [12]:
data_with_egde_val

,H2,CO,C2H4,C2H2,H2v,COv,C2H4v,C2H2v,H2a,COa,C2H4a,C2H2a,status
0,0.002294,0.042099,0.004630,0.000345,0.000004,0.000075,0.000020,5.818826e-07,-2.304700e-09,1.774780e-07,1.974990e-08,5.403102e-10,1.0
1,0.002764,0.037128,0.006520,0.000146,0.000005,0.000054,0.000033,6.528160e-07,-4.561590e-08,-4.215120e-07,-1.257772e-07,-4.105670e-09,1.0
2,0.001816,0.036885,0.002999,0.000253,0.000006,0.000087,0.000022,1.189219e-06,-1.104300e-08,-3.439780e-07,-2.257930e-08,-2.247120e-09,1.0
3,0.001659,0.028044,0.008604,0.000310,0.000006,0.000062,0.000019,1.690292e-07,-3.560000e-09,-4.741500e-08,-1.998480e-08,3.894702e-10,1.0
4,0.002091,0.014124,0.013251,0.000347,0.000005,0.000044,0.000029,8.600990e-07,5.168470e-08,2.986550e-07,4.467500e-08,3.707080e-09,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.002149,0.011889,0.006705,0.000209,0.000007,0.000021,0.000026,6.132419e-07,3.950630e-08,3.650550e-07,7.098680e-08,5.182260e-09,1.0
2096,0.001105,0.035809,0.007522,0.000368,0.000002,0.000079,0.000026,1.244349e-06,-3.644930e-08,-5.906600e-07,-6.026740e-08,-8.906090e-09,1.0
2097,0.004339,0.031754,0.004703,0.000291,0.000012,0.000109,0.000022,3.293170e-07,1.763760e-08,5.331860e-07,1.371743e-07,3.483340e-09,1.0
2098,0.003762,0.022104,0.004537,0.000454,0.000006,0.000070,0.000024,1.327107e-06,-2.584230e-08,-3.851510e-07,-1.212187e-07,-1.476280e-09,4.0


In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

In [14]:
data_full = data_with_egde_val.copy()
X_data = data_full.drop('status', axis=1)
y = data_full.status

In [15]:
seed = 7
test_size = 0.3
X_trian, X_test, y_train, y_test = train_test_split(X_data, y, test_size=test_size, random_state=seed)

In [16]:

param_test = {
 'max_depth':[2,3,4,5,6,7,8],
 'min_child_weight':[2,3,4,5,6,7,8],
  'gamma':np.linspace(0, 1, num = 5),
    'subsample':np.linspace(0, 1, num = 5),
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, colsample_bytree=0.8,
  nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring=f1,n_jobs=24,iid=False, cv=5,verbose=8)

model = gsearch.fit(X_trian, y_train)
pred = model.predict(X_test)
print("Accuracy for model: %.2f" % (f1_score(y_test, pred,average='macro')))

Fitting 5 folds for each of 1225 candidates, totalling 6125 fits


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done  24 tasks      | elapsed:    2.7s
[Parallel(n_jobs=24)]: Done  65 tasks      | elapsed:    4.9s
[Parallel(n_jobs=24)]: Done 114 tasks      | elapsed:    7.3s
[Parallel(n_jobs=24)]: Done 173 tasks      | elapsed:    9.9s
[Parallel(n_jobs=24)]: Done 240 tasks      | elapsed:   14.3s
[Parallel(n_jobs=24)]: Done 317 tasks      | elapsed:   18.5s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:   23.9s
[Parallel(n_jobs=24)]: Done 497 tasks      | elapsed:   29.4s
[Parallel(n_jobs=24)]: Done 600 tasks      | elapsed:   36.2s
[Parallel(n_jobs=24)]: Done 713 tasks      | elapsed:   43.5s
[Parallel(n_jobs=24)]: Done 834 tasks      | elapsed:   51.7s
[Parallel(n_jobs=24)]: Done 965 tasks      | elapsed:  1.0min
[Parallel(n_jobs=24)]: Done 1104 tasks      | elapsed:  1.2min
[Parallel(n_jobs=24)]: Done 1253 tasks      | elapsed:  1.3min
[Parallel(n_jobs=24)]: Done 1410 tasks      | elapsed

[19:10:02] WARNING: ../src/learner.cc:541: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:10:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy for model: 0.86


EVO

In [20]:
data_evo = funcs_for_proc.get_data_from_csv('class/data_test/data_test', 'csv',False)

In [23]:
data_with_egde_val_evo=pd.DataFrame(columns=['filename','H2', 'CO', 'C2H4', 'C2H2','H2v', 'COv', 'C2H4v', 'C2H2v','H2a', 'COa', 'C2H4a', 'C2H2a'])
data_for_an_evo=data_evo
for i,file in enumerate(data_for_an_evo):
    insertdata=[]
    datav=file[2].diff()
    dataa=datav.diff()
    vmax_last=list(datav.iloc[len(datav)-1].values)
    amax_last=list(dataa.iloc[len(dataa)-1].values)
    vmax_last=list(datav.iloc[len(datav)-1].values)
    con=list(file[2].iloc[len(file[2])-1].values)
    insertdata=[file[0]]+con+vmax_last+amax_last
    data_with_egde_val_evo.loc[i]=insertdata
    
    

In [24]:
data_with_egde_val_evo.head()

,filename,H2,CO,C2H4,C2H2,H2v,COv,C2H4v,C2H2v,H2a,COa,C2H4a,C2H2a
0,2_trans_88.csv,0.000819,0.007002,0.006456,0.000555,0.000003,0.000044,0.000014,1.213661e-06,5.744786e-08,2.956775e-07,1.402536e-07,1.982610e-09
1,2_trans_463.csv,0.003013,0.029694,0.010937,0.000320,0.000012,0.000067,0.000019,1.347287e-06,5.385600e-09,4.570690e-07,2.746700e-08,2.079970e-09
2,2_trans_2619.csv,0.001329,0.013145,0.002032,0.000410,0.000003,0.000060,0.000007,6.467486e-07,2.888380e-08,3.254910e-07,4.126940e-08,2.041650e-09
3,2_trans_470.csv,0.003533,0.017728,0.006918,0.000353,0.000006,0.000036,0.000013,5.268020e-07,2.091760e-08,5.543680e-07,1.543874e-07,4.614700e-09
4,2_trans_2785.csv,0.002487,0.041149,0.010973,0.000325,0.000006,0.000094,0.000024,4.847960e-07,2.322770e-08,1.757400e-07,9.658000e-08,4.345080e-09


In [25]:
data_full_evo = data_with_egde_val_evo.copy()
X_data_test_evo = data_full_evo.drop('filename', axis=1)


In [26]:
pred_evo = model.predict(X_data_test_evo)

In [27]:
exportdata=pd.DataFrame([data_full_evo.filename,pred_evo.astype(int)]).T

In [28]:
exportdata.head()

,filename,Unnamed 0
0,2_trans_88.csv,3
1,2_trans_463.csv,1
2,2_trans_2619.csv,1
3,2_trans_470.csv,1
4,2_trans_2785.csv,1


In [29]:
exportdata.to_csv('results_evo/xgb_mod_v2.csv',header=['id','category'],index=False)